# 06 Eye-Tracking Event Analysis

## Overview
This notebook analyzes **eye-tracking data** from SR Research EyeLink recordings, synchronizing gaze behavior with experimental events.

**Purpose:**
- Load and parse EyeLink EDF files containing gaze coordinates and pupil size measurements
- Synchronize eye-tracking timestamps with experimental stimulus presentation (AI_Response.started events)
- Extract eye-tracking features for each trial to analyze reading behavior and cognitive load

**What it does:**
1. Loads session mapping (from Notebook 01) to identify which sessions have eye-tracking data
2. For each matched session:
   - Parses the corresponding EDF file to extract gaze position (X, Y coordinates) and pupil sizes
   - Synchronizes eye-tracking timestamps with experiment event timestamps
   - Segments eye-tracking data by trial based on AI_Response.started markers
   - Validates gaze positions within the defined Region of Interest (ROI) for text stimuli
3. Computes eye-tracking features per trial:
   - **Fixations:** duration, count, spatial dispersion, time to first fixation
   - **Saccades:** amplitude, duration, frequency
   - **Pupil metrics:** mean size, Index of Pupillary Activity (IPA)
4. Saves extracted features for statistical analysis

**Output:**
- `session_00-eye-features.pkl`, `session_01-eye-features.pkl`, etc.
- Each file contains trial-level eye-tracking features ready for analysis

**Next step:** Use features to analyze differences across alignment conditions or correlate with EEG/behavioral measures.

**Note:** This notebook requires preprocessing of EDF files. Install `pyedfread` or use MNE for EDF parsing.

## 1. Import Libraries and Setup

In [2]:
import numpy as np
import pandas as pd
import glob

from matplotlib import pyplot as plt
import matplotlib.patches as patches


from tqdm.notebook import tqdm

In [3]:
## 2. Load Session Mapping

# Load session mapping for eye-tracking data
df_sessions = pd.read_csv('./session_mapping.csv')

# Filter sessions with EEG match (eye-tracking available)
df_matched = df_sessions[df_sessions['eeg_file'] != 'NO MATCH'].copy()

# Extract session IDs
session_ids = df_matched.index.tolist()
print(f'Available sessions: {session_ids}')
print(f'Number of sessions: {len(session_ids)}')

# Show session info
print('\nSession details:')
print(df_matched[['experiment_file', 'n_trials', 'coverage']])

Available sessions: [0, 1, 4]
Number of sessions: 3

Session details:
                                     experiment_file  n_trials coverage
0  01_human-llm-alignment_2025-11-17_11h36.44.912...        50    60.3%
1  01_human-llm-alignment_2025-11-20_13h16.37.791...        39    79.3%
4  01_human-llm-alignment_2025-11-24_14h13.05.529...        49    54.7%


## 3. Define Region of Interest (ROI)

The ROI defines the screen area where text stimuli are presented. Gaze positions within this region indicate reading behavior.

In [4]:
# Region of Interest (ROI) for text stimulus
# TODO: Adjust to actual stimulus coordinates!
x1 = 585
y1 = 165.0

x2 = 1335
y2 = 915.0

roi_width = x2 - x1
roi_height = y2 - y1
print(f'ROI size: {roi_width} x {roi_height} pixels')
print(f'ROI coordinates: ({x1}, {y1}) to ({x2}, {y2})')

ROI size: 750 x 750.0 pixels
ROI coordinates: (585, 165.0) to (1335, 915.0)


## 4. Load EDF Files

EyeLink stores eye-tracking data in EDF (European Data Format) files. We'll use MNE or pyedfread to parse these files.

In [5]:
# Check available EDF files
from pathlib import Path

edf_files = list(Path('./Data').glob('*.EDF'))
print(f'Found {len(edf_files)} EDF files:')
for edf in edf_files:
    print(f'  {edf.name}')

# Map EDF files to sessions
# EDF files should correspond to the EEG files in session_mapping
print('\nMapping EDF files to sessions:')
for idx, row in df_matched.iterrows():
    # Try to find corresponding EDF file
    # EDF naming might differ - check multiple patterns
    eeg_timestamp = row['eeg_file'].replace('EEG_data_', '').replace('.csv', '')
    
    # Look for matching EDF files
    matching_edfs = [f for f in edf_files if eeg_timestamp in f.name or 
                     row['experiment_date'][:10].replace('-', '_') in f.name]
    
    if matching_edfs:
        print(f"Session {idx}: {matching_edfs[0].name}")
    else:
        print(f"Session {idx}: No matching EDF found")
        print(f"  Looking for timestamp: {eeg_timestamp}")
        print(f"  Experiment date: {row['experiment_date']}")

Found 10 EDF files:
  201101_2025_11_20_13_16.EDF
  201102_2025_11_20_15_02 copy.EDF
  201102_2025_11_20_15_02.EDF
  201105_2025_11_20_16_25.EDF
  241101_2025_11_24_14_13.EDF
  271103_2025_11_27_10_49.EDF
  271105_2025_11_27_12_50.EDF
  2711ih_2025_11_27_11_29 copy.EDF
  2711ih_2025_11_27_11_29.EDF
  tb171106_2025_11_17_11_36.EDF

Mapping EDF files to sessions:
Session 0: tb171106_2025_11_17_11_36.EDF
Session 1: 201101_2025_11_20_13_16.EDF
Session 4: 241101_2025_11_24_14_13.EDF


## 5. Example: Parse Single EDF File

Test loading one EDF file to understand the data structure.

In [6]:
# Function to parse EDF file - Try multiple methods
def parse_edf_file(edf_path):
    """
    Parse EyeLink EDF file and extract eye-tracking data.
    Tries multiple parsing methods.
    """
    from pathlib import Path
    
    # Method 1: Try pyedfread (if available)
    try:
        import pyedfread
        samples, events, messages = pyedfread.edf.pread(str(edf_path))
        print(f"Successfully parsed with pyedfread: {Path(edf_path).name}")
        print(f"Samples: {len(samples)} rows")
        print(f"Events: {len(events)} events")
        print(f"Available columns: {samples.columns.tolist()}")
        return samples, events, messages
    except ImportError:
        print("pyedfread not available")
    except Exception as e:
        print(f"pyedfread error: {e}")
    
    # Method 2: Try converting to ASC first
    try:
        import subprocess
        asc_path = str(edf_path).replace('.EDF', '.asc')
        
        # Check if ASC file already exists
        if not Path(asc_path).exists():
            print(f"EDF file needs to be converted to ASC format")
            print(f"Use SR Research's edf2asc converter:")
            print(f"  edf2asc {edf_path}")
            print(f"\nOr use DataViewer to export data")
            return None, None, None
        else:
            # Parse ASC file
            print(f"Found ASC file: {Path(asc_path).name}")
            df_samples = parse_asc_file(asc_path)
            return df_samples, None, None
            
    except Exception as e:
        print(f"ASC parsing error: {e}")
    
    return None, None, None

def parse_asc_file(asc_path):
    """
    Parse EyeLink ASC (ASCII) file.
    """
    samples = []
    
    with open(asc_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            
            # Sample lines start with a timestamp (number)
            if len(parts) > 0 and parts[0].isdigit():
                try:
                    timestamp = float(parts[0])
                    
                    # Format: timestamp x_left y_left pupil_left x_right y_right pupil_right
                    if len(parts) >= 7:
                        samples.append({
                            'timestamp': timestamp,
                            'x_left': float(parts[1]) if parts[1] != '.' else np.nan,
                            'y_left': float(parts[2]) if parts[2] != '.' else np.nan,
                            'pupil_left': float(parts[3]) if parts[3] != '.' else np.nan,
                            'x_right': float(parts[4]) if parts[4] != '.' else np.nan,
                            'y_right': float(parts[5]) if parts[5] != '.' else np.nan,
                            'pupil_right': float(parts[6]) if parts[6] != '.' else np.nan
                        })
                except (ValueError, IndexError):
                    continue
    
    df = pd.DataFrame(samples)
    df['timestamp'] = df['timestamp'] / 1000.0  # Convert ms to seconds
    
    print(f"Parsed {len(df)} samples from ASC file")
    return df

# Test with first EDF file
if len(edf_files) > 0:
    test_edf = edf_files[0]
    print(f"\nTesting EDF parsing with: {test_edf.name}")
    samples, events, messages = parse_edf_file(test_edf)
    
    if samples is not None and len(samples) > 0:
        df_test = samples if isinstance(samples, pd.DataFrame) else pd.DataFrame(samples)
        print(f"\nExtracted columns: {df_test.columns.tolist()}")
        print(f"\nFirst few rows:")
        print(df_test.head())
        print(f"\nData summary:")
        print(df_test.describe())
    else:
        print("\n" + "="*60)
        print("EDF files need to be converted to ASC format first!")
        print("="*60)
        print("\nOptions:")
        print("1. Use SR Research edf2asc converter (Windows):")
        print("   Download from: https://www.sr-research.com/support/")
        print("   Run: edf2asc.exe <filename>.EDF")
        print("\n2. Use EyeLink DataViewer to export data")
        print("\n3. Install pyedfread (Linux/Mac):")
        print("   pip install pyedfread")


Testing EDF parsing with: 201101_2025_11_20_13_16.EDF
pyedfread not available
EDF file needs to be converted to ASC format
Use SR Research's edf2asc converter:
  edf2asc Data\201101_2025_11_20_13_16.EDF

Or use DataViewer to export data

EDF files need to be converted to ASC format first!

Options:
1. Use SR Research edf2asc converter (Windows):
   Download from: https://www.sr-research.com/support/
   Run: edf2asc.exe <filename>.EDF

2. Use EyeLink DataViewer to export data

3. Install pyedfread (Linux/Mac):
   pip install pyedfread


### Alternative: Try reading EDF as binary

Since standard tools aren't working, let's try reading the EDF header to understand the format.

In [7]:
# Read EDF file header to check format
test_edf = edf_files[0]

with open(test_edf, 'rb') as f:
    # Read first 256 bytes (header)
    header = f.read(256)
    
    # Try to decode as ASCII
    try:
        header_text = header.decode('ascii', errors='ignore')
        print("EDF Header:")
        print(header_text[:200])
    except:
        print("Binary header - EyeLink proprietary format")
        print(f"First bytes: {header[:50]}")

print("\n" + "="*60)
print("SOLUTION: Use SR Research edf2asc converter")
print("="*60)
print("\nDownload link:")
print("https://www.sr-research.com/support/thread-13.html")
print("\nAfter installing, run this in PowerShell:")
print("cd Data")
print("edf2asc -t -s -miss -1.0 -y *.EDF")
print("\nThis will create .asc files that we can parse")

EDF Header:
SR_RESEARCH_1000FILE
DATE: Thu Nov 20 13:23:42 2025
TYPE: EDF_FILE BINARY EVENT SAMPLE TAGGED
VERSION: EYELINK II 1
SOURCE: EYELINK CL
EYELINK II CL v5.50 Jun 16 2022 (EyeLink 1000 Plus)
CAMERA: Eyeli

SOLUTION: Use SR Research edf2asc converter

Download link:
https://www.sr-research.com/support/thread-13.html

After installing, run this in PowerShell:
cd Data
edf2asc -t -s -miss -1.0 -y *.EDF

This will create .asc files that we can parse


## 5.1 Synchronize Eye-Tracking with Experiment Events

Match eye-tracking timestamps with experimental trial timing.

In [28]:
def synchronize_eye_with_experiment(df_eye, experiment_file, session_id):
    """
    Synchronize eye-tracking data with experiment events.
    
    Parameters:
    -----------
    df_eye : pd.DataFrame
        Eye-tracking data with timestamp column
    experiment_file : str
        Path to experiment CSV file
    session_id : int
        Session ID for tracking
        
    Returns:
    --------
    pd.DataFrame with eye-tracking data segmented by trials
    """
    # Load experiment data
    df_exp = pd.read_csv(f'./Data/{experiment_file}')
    
    # Extract AI_Response.started timestamps (in seconds from experiment start)
    # These mark when each text stimulus was presented
    ai_response_times = df_exp['AI_Response.started'].dropna().values
    
    # Also get trial info
    trial_info = df_exp[df_exp['AI_Response.started'].notna()].copy()
    trial_info = trial_info.reset_index(drop=True)
    
    print(f"Found {len(ai_response_times)} trials in experiment")
    print(f"Eye-tracking duration: {df_eye['timestamp'].max():.2f} seconds")
    print(f"Experiment duration: {ai_response_times.max():.2f} seconds")
    
    # Calculate offset between eye-tracking and experiment
    # This assumes both started at roughly the same time
    # You may need to adjust based on your actual synchronization method
    
    # Option 1: Assume they start at the same time
    time_offset = 0
    
    # Option 2: Use session_mapping offset
    # time_offset = df_matched.loc[session_id, 'time_offset_min'] * 60  # convert to seconds
    
    # Segment eye-tracking data by trial
    trial_segments = []
    
    for i, trial_start in enumerate(ai_response_times):
        # Get trial end (next trial start or end of data)
        if i < len(ai_response_times) - 1:
            trial_end = ai_response_times[i + 1]
        else:
            trial_end = trial_start + 30  # Assume max 30 seconds for last trial
        
        # Extract eye-tracking data for this trial
        mask = (df_eye['timestamp'] >= trial_start + time_offset) & \
               (df_eye['timestamp'] < trial_end + time_offset)
        
        df_trial_eye = df_eye[mask].copy()
        df_trial_eye['trial_number'] = i
        df_trial_eye['trial_time'] = df_trial_eye['timestamp'] - (trial_start + time_offset)
        
        # Add trial metadata
        if i < len(trial_info):
            for col in ['ScenarioLoop.thisN', 'AI_Alignment', 'text.text']:
                if col in trial_info.columns:
                    df_trial_eye[col] = trial_info.loc[i, col]
        
        trial_segments.append(df_trial_eye)
        
        # Check if gaze was within ROI
        if len(df_trial_eye) > 0:
            # Use left eye by default, or combine both
            x_gaze = df_trial_eye['x_left'].fillna(df_trial_eye['x_right'])
            y_gaze = df_trial_eye['y_left'].fillna(df_trial_eye['y_right'])
            
            in_roi = ((x_gaze > x1) & (x_gaze < x2) & 
                      (y_gaze > y1) & (y_gaze < y2))
            
            roi_percentage = in_roi.sum() / len(df_trial_eye) * 100
            
            if roi_percentage < 50:
                print(f"Trial {i}: Low ROI coverage ({roi_percentage:.1f}%)")
    
    # Combine all trials
    df_eye_trials = pd.concat(trial_segments, ignore_index=True)
    
    print(f"\nSegmented {len(trial_segments)} trials")
    print(f"Total eye-tracking samples: {len(df_eye_trials)}")
    
    return df_eye_trials, trial_info

# Test synchronization with first session
if len(edf_files) > 0 and 'df_test' in locals():
    session_id = session_ids[0]
    exp_file = df_matched.loc[session_id, 'experiment_file']
    
    print(f"\nTesting synchronization for Session {session_id}")
    print(f"Experiment file: {exp_file}")
    
    try:
        df_synced, trial_info = synchronize_eye_with_experiment(df_test, exp_file, session_id)
        print(f"\nSynchronized data shape: {df_synced.shape}")
        print(f"Columns: {df_synced.columns.tolist()}")
        print(f"\nSample of synchronized data:")
        print(df_synced.head(10))
    except Exception as e:
        print(f"Error during synchronization: {e}")
        import traceback
        traceback.print_exc()

## 5.2 Process All Sessions

Extract and save eye-tracking data for all available sessions.

In [29]:
def process_all_sessions_eye_tracking():
    """
    Process eye-tracking data for all sessions with EDF files.
    """
    all_eye_data = []
    
    for session_id in tqdm(session_ids, desc="Processing sessions"):
        try:
            # Get experiment file
            exp_file = df_matched.loc[session_id, 'experiment_file']
            
            # Find corresponding EDF file
            # Try multiple matching strategies
            eeg_timestamp = df_matched.loc[session_id, 'eeg_file'].replace('EEG_data_', '').replace('.csv', '')
            exp_date = df_matched.loc[session_id, 'experiment_date'][:10].replace('-', '_')
            
            # Look for matching EDF
            matching_edfs = [f for f in edf_files if eeg_timestamp in f.name or exp_date in f.name]
            
            if not matching_edfs:
                print(f"Session {session_id}: No EDF file found")
                continue
            
            edf_file = matching_edfs[0]
            print(f"\nSession {session_id}: Processing {edf_file.name}")
            
            # Parse EDF
            df_eye, info = parse_edf_file(edf_file)
            
            # Synchronize with experiment
            df_synced, trial_info = synchronize_eye_with_experiment(df_eye, exp_file, session_id)
            
            # Add session ID
            df_synced['session_id'] = session_id
            
            # Save to preprocessed folder
            output_path = f'./preprocessed/session_{session_id:02d}-eye.pkl'
            df_synced.to_pickle(output_path)
            print(f"Saved to {output_path}")
            
            all_eye_data.append(df_synced)
            
        except Exception as e:
            print(f"Error processing session {session_id}: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    print(f"\n{'='*60}")
    print(f"Successfully processed {len(all_eye_data)} sessions")
    
    return all_eye_data

# Run processing (uncomment when ready)
# all_eye_data = process_all_sessions_eye_tracking()

In [30]:
# Combine all sessions
# df = pd.concat(dfs, keys=session_ids, names=['SessionID'])
# df = df.reset_index()

In [31]:
# Check for invalid trials (no gaze within ROI)
# df[df.Valid == False].SessionID.value_counts()

In [32]:
# Check trials with no fixation time
# df[df.TimeFixation == 0].SessionID.value_counts()

In [33]:
# Filter for valid fixations
# dfX = df[df.TimeFixation != 0]

In [34]:
# Plot distribution of time to first fixation
# plt.figure(figsize=(10, 6))
# plt.hist(dfX.TimeFixation - dfX.TimeStart, bins=50, range=(0, 2000))
# plt.xlabel('Time to First Fixation (ms)')
# plt.ylabel('Count (log scale)')
# plt.title('Distribution of Time to First Fixation')
# plt.yscale('log')
# plt.show()

## 6. Visualization

Visualize gaze patterns and fixations overlaid on stimulus presentation area.

In [35]:
# Import for EDF file parsing
# Option 1: pyedfread (for SR Research EyeLink)
# pip install pyedfread

# Option 2: MNE (also has EDF support)
import mne

# Example: Load an EDF file
edf_path = './Data/201101_2025_11_20_13_16.EDF'

try:
    # MNE can read EDF files (primarily for biosignals, but works)
    raw_edf = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)
    print(f"EDF file loaded: {edf_path}")
    print(f"Channels: {raw_edf.ch_names}")
    print(f"Sampling rate: {raw_edf.info['sfreq']} Hz")
    print(f"Duration: {raw_edf.times[-1]:.2f} seconds")
except Exception as e:
    print(f"Error loading: {e}")
    print("\nAlternative: Install pyedfread for better EyeLink support")
    print("pip install pyedfread")

Error loading: Bad EDF file provided.

Alternative: Install pyedfread for better EyeLink support
pip install pyedfread


C:\Users\kimbe\AppData\Local\Temp\ipykernel_20196\2385035987.py:13: RuntimeWarning: Invalid measurement date encountered in the header.
  raw_edf = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)


In [36]:
# Visualize gaze pattern for a single trial
def plot_trial_gaze(df_trial, trial_num, show_roi=True):
    """
    Plot gaze trajectory for a single trial.
    """
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot gaze trajectory
    x_gaze = df_trial['x_left'].fillna(df_trial['x_right'])
    y_gaze = df_trial['y_left'].fillna(df_trial['y_right'])
    
    # Color by time
    scatter = ax.scatter(x_gaze, y_gaze, c=df_trial['trial_time'], 
                        cmap='viridis', s=5, alpha=0.6)
    
    # Mark start and end
    ax.plot(x_gaze.iloc[0], y_gaze.iloc[0], 'go', markersize=10, label='Start')
    ax.plot(x_gaze.iloc[-1], y_gaze.iloc[-1], 'ro', markersize=10, label='End')
    
    # Draw ROI
    if show_roi:
        rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                                linewidth=2, edgecolor='r', facecolor='none',
                                label='Text ROI')
        ax.add_patch(rect)
    
    # Set up plot
    ax.set_xlim(0, 1920)
    ax.set_ylim(1080, 0)  # Flip Y-axis (screen coordinates)
    ax.set_xlabel('X Position (pixels)')
    ax.set_ylabel('Y Position (pixels)')
    ax.set_title(f'Gaze Pattern - Trial {trial_num}')
    ax.legend()
    ax.set_aspect('equal')
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('Time in trial (s)')
    
    plt.tight_layout()
    plt.show()

# Example: Plot first trial if data is available
# if 'df_synced' in locals():
#     df_trial_0 = df_synced[df_synced['trial_number'] == 0]
#     if len(df_trial_0) > 0:
#         plot_trial_gaze(df_trial_0, 0)

In [37]:
# Next steps:

# 1. Parse EDF files and convert to DataFrame
#    - Extract gaze coordinates (X, Y)
#    - Extract pupil sizes
#    - Synchronize timestamps

# 2. Synchronize with experiment data
#    - Load corresponding CSV from Data/
#    - Find AI_Response.started timestamps
#    - Segment eye-tracking data for each trial

# 3. Compute features
#    - Detect fixations
#    - Analyze saccades
#    - Calculate pupil size per trial
#    - ROI analysis (time in ROI, first fixation, etc.)

print('Notebook template created. Adapt it to your EDF data!')

Notebook template created. Adapt it to your EDF data!


In [38]:
# Optional: Check other files
# (Commented out - only as reference)

## Feature Computation

In [39]:
# Example: Feature computation for one session
# When eye-tracking data is preprocessed:

# session_id = 0  # First session
# eye_data_df = pd.read_pickle(f'./preprocessed/session_{session_id:02d}-eye.pkl')
# experiment_df = pd.read_csv(df_matched.loc[session_id, 'experiment_file'])

# Show available columns
# print('Eye-tracking columns:', eye_data_df.columns.tolist())
# print('Experiment columns:', experiment_df.columns.tolist())

In [40]:
import pandas as pd
import numpy as np
from scipy import stats, signal
from scipy.spatial import ConvexHull
import pywt

ModuleNotFoundError: No module named 'pywt'

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull
from tqdm import tqdm
import pywt

def calculate_modmax(d):
    """Calculate modulus maxima for IPA calculation"""
    m = [abs(x) for x in d]
    t = [0.0] * len(d)
    
    for i in range(len(d)):
        ll = m[i-1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i+1] if i < len(d)-2 else m[i]
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
            t[i] = np.sqrt(d[i]**2)
        else:
            t[i] = 0.0
    return t

def calculate_ipa(d, t):
    """Calculate Index of Pupillary Activity (IPA)"""
    # Handle cases with insufficient data
    if len(d) < 2:
        return 0
    
    try:
        # Use max decomposition level based on data length
        max_level = pywt.dwt_max_level(len(d), 'sym8')
        level = min(2, max_level)
        
        # Wavelet decomposition with adaptive level
        coeffs = pywt.wavedec(d, 'sym8', 'periodic', level=level)
        
        # Unpack coefficients based on the decomposition level
        if level == 2:
            cA2, cD2, cD1 = coeffs[:3]
        else:
            cA2, cD2 = coeffs
            cD1 = np.zeros(1)  # Placeholder
    except ValueError as e:
        print(f"IPA calculation error: {e}")
        return 0
    
    # Normalize coefficients
    cA2 = [x / np.sqrt(4.0) for x in cA2]
    cD1 = [x / np.sqrt(2.0) for x in cD1]
    cD2 = [x / np.sqrt(4.0) for x in cD2]
    
    # Calculate modulus maxima
    cD2m = calculate_modmax(cD2)
    
    # Thresholding
    lambda_univ = np.std(cD2m) * np.sqrt(2.0 * np.log2(len(cD2m)))
    cD2t = pywt.threshold(cD2m, lambda_univ, mode="hard")
    
    # Calculate IPA
    ctr = sum(1 for x in cD2t if abs(x) > 0)
    tt = t[-1] - t[0]
    IPA = float(ctr) / tt if tt > 0 else 0
    
    return IPA

def calculate_fixations(df, velocity_threshold=20, min_fixation_duration=30):
    """Calculate fixations with more lenient parameters and robust time handling"""
    if len(df) < 2:
        return df, []
    
    df = df.copy()
    
    # 1. Better timestamp handling
    # Use timestamps relative to trial start to avoid precision issues
    df['time_ms'] = (df['UnixTime'] - df['UnixTime'].iloc[0])
    
    # Instead of interpolating zero differences, take cumulative time
    # This ensures we always have increasing timestamps
    df['time_ms'] = np.arange(len(df)) * (1000/1000)  # Assuming 1000Hz sampling
    
    # 2. Robust coordinate handling
    # Forward fill any NaN values in coordinates
    df['X_L'] = df['X_L'].fillna(method='ffill').fillna(method='bfill')
    df['Y_L'] = df['Y_L'].fillna(method='ffill').fillna(method='bfill')
    
    # 3. Better velocity calculation
    # Calculate velocities using rolling windows to reduce noise
    window_size = 3
    dx = df['X_L'].rolling(window=window_size, center=True).mean().diff()
    dy = df['Y_L'].rolling(window=window_size, center=True).mean().diff()
    dt = df['time_ms'].diff() / 1000.0  # Convert to seconds
    
    dt = dt.replace(0, 1/1000)  # Use sampling rate for zero differences
    
    # Calculate velocities (degrees per second)
    velocities = np.sqrt(dx**2 + dy**2) / dt
    velocities = velocities.fillna(method='ffill').fillna(method='bfill')
    velocities = np.clip(velocities, 0, 1000)  # Clip extreme values
    
    df['velocity'] = velocities
    df['is_fixation'] = df['velocity'] < velocity_threshold
    
    window_size = 5  # 5ms at 1000Hz
    df['is_fixation_smooth'] = df['is_fixation'].rolling(window=window_size, center=True, min_periods=1).mean() > 0.5
    df['fixation_group'] = (df['is_fixation_smooth'] != df['is_fixation_smooth'].shift()).cumsum()
    
    # Process fixation groups
    valid_groups = []
    group_info = []
    
    for group_id, group in df[df['is_fixation_smooth']].groupby('fixation_group'):
        duration = (group['time_ms'].max() - group['time_ms'].min()) / 1000.0  # Convert to seconds
        
        # Only keep fixations that meet minimum duration
        if duration >= min_fixation_duration/1000.0:
            valid_groups.append(group_id)
            group_info.append({
                'group_id': group_id,
                'duration': duration,
                'x_mean': group['X_L'].mean(),
                'y_mean': group['Y_L'].mean()
            })
    
    df['is_valid_fixation'] = df['is_fixation_smooth'] & df['fixation_group'].isin(valid_groups)
    df['fixation_group'] = np.where(df['is_valid_fixation'], df['fixation_group'], 0)
    
    # Print summary only if we found fixations
    if len(valid_groups) > 0:
        durations = [g['duration'] for g in group_info]
        #print(f"Found {len(valid_groups)} fixations. Mean duration: {np.mean(durations):.3f}s")
    
    return df, group_info

def process_participant_data(pid, base_path="../preprocessed/"):
    """Process participant data with improved fixation detection"""
    eye_data = pd.read_pickle(f"{base_path}{pid}-eye.pkl")
    events = pd.read_pickle(f"{base_path}{pid}-stats.pkl")
    
    print(f"Processing participant {pid}")
    
    all_features = []
    total_fixations = 0
    
    for _, event in events.iterrows():
        trial_data = eye_data[
            (eye_data['UnixTime'] >= event['TimeStart']) & 
            (eye_data['UnixTime'] <= event['TimeEnd'])
        ].copy()
        
        if len(trial_data) < 2:
            continue

        features = {}
        
        # Calculate pupil sizes with validity check
        valid_left = trial_data['PupilSize_L'].notna() & (trial_data['PupilSize_L'] > 0)
        valid_right = trial_data['PupilSize_R'].notna() & (trial_data['PupilSize_R'] > 0)
        
        left_values = trial_data.loc[valid_left, 'PupilSize_L']
        right_values = trial_data.loc[valid_right, 'PupilSize_R']
        
        features['leftPupilSize'] = np.nanmean(left_values) if len(left_values) > 0 else 0
        features['rightPupilSize'] = np.nanmean(right_values) if len(right_values) > 0 else 0

        # Initialize fixation features
        features.update({
            'fixationDur_mean': 0, 'fixationDur_var': 0, 'fixationDur_std': 0,
            'fixationDur_total': 0, 'fixationCount': 0, 'fixationsRate': 0,
            'time2FirstFixation': 0, 'firstFixationDur': 0, 'time2LastFixation': 0,
            'lastFixationDur': 0, 'fixationDispersionArea': 0, 'fixationSlope': 0,
            'saccadeAmp': 0, 'saccadeDur': 0, 'saccadeFreq': 0
        })

        # Calculate fixations
        trial_data, fixation_info = calculate_fixations(trial_data)
        
        if fixation_info:  # If we have valid fixations
            durations = [g['duration'] for g in fixation_info]
            features.update({
                'fixationDur_mean': np.mean(durations),
                'fixationDur_var': np.var(durations),
                'fixationDur_std': np.std(durations),
                'fixationDur_total': np.sum(durations),
                'fixationCount': len(fixation_info),
                'fixationsRate': len(fixation_info) / ((event['TimeEnd'] - event['TimeStart']) / 1000.0),
                'time2FirstFixation': (fixation_info[0]['group_id'] - event['TimeStart']) / 1000.0,
                'firstFixationDur': durations[0],
                'time2LastFixation': (fixation_info[-1]['group_id'] - event['TimeStart']) / 1000.0,
                'lastFixationDur': durations[-1]
            })

            # Calculate spatial features if we have enough fixations
            fix_points = np.array([[g['x_mean'], g['y_mean']] for g in fixation_info])
            
            if len(fix_points) >= 3:
                try:
                    hull = ConvexHull(fix_points)
                    features['fixationDispersionArea'] = hull.area
                except Exception as e:
                    print(f"Convex hull error: {e}")

            if len(fix_points) > 1:
                try:
                    features['fixationSlope'] = np.polyfit(fix_points[:, 0], fix_points[:, 1], 1)[0]
                except Exception as e:
                    print(f"Polyfit error: {e}")

        # Calculate saccade features
        saccades = trial_data[~trial_data['is_valid_fixation']]
        if len(saccades) > 0:
            features['saccadeAmp'] = np.nanmean(saccades['velocity'])
            features['saccadeDur'] = len(saccades) * np.mean(np.diff(trial_data['time_ms'])) / 1000.0
            total_time = (event['TimeEnd'] - event['TimeStart']) / 1000.0
            features['saccadeFreq'] = len(saccades) / total_time if total_time > 0 else 0

        # Calculate IPA
        if event['TaskCount'] > 0:
            prev_event = events[events['TaskCount'] == event['TaskCount'] - 1].iloc[0]
            baseline_data = eye_data[
                (eye_data['UnixTime'] >= prev_event['TimeEnd'] - 2000) & 
                (eye_data['UnixTime'] <= prev_event['TimeEnd'])
            ]
            if len(baseline_data) > 0:
                left_baseline = np.nanmean(baseline_data['PupilSize_L'])
                right_baseline = np.nanmean(baseline_data['PupilSize_R'])
            else:
                left_baseline = right_baseline = 0
        else:
            left_baseline = right_baseline = 0

        left_pupil = trial_data['PupilSize_L'].fillna(method='ffill').values - left_baseline
        right_pupil = trial_data['PupilSize_R'].fillna(method='ffill').values - right_baseline
        time_vals = (trial_data['UnixTime'] - event['TimeStart']).values / 1000.0

        try:
            features['leftIPA'] = calculate_ipa(left_pupil, time_vals) if len(left_pupil) > 0 else 0
        except Exception as e:
            print(f"Left IPA calculation error: {e}")
            features['leftIPA'] = 0

        try:
            features['rightIPA'] = calculate_ipa(right_pupil, time_vals) if len(right_pupil) > 0 else 0
        except Exception as e:
            print(f"Right IPA calculation error: {e}")
            features['rightIPA'] = 0

        features['PId'] = event['PId']
        features['TaskCount'] = event['TaskCount']
        features['Condition'] = event['Condition']
        features['Rating'] = event['Rating']

        all_features.append(features)

    features_df = pd.DataFrame(all_features)
    features_df.to_pickle(f"{base_path}{pid}-features.pkl")
    
    return features_df

def process_all_participants(pids, base_path="../preprocessed/"):
    """Process all participants"""
    results = {}
    failed_pids = []
    
    for pid in tqdm(pids, desc="Processing participants"):
        try:
            features_df = process_participant_data(pid, base_path)
            results[pid] = features_df
            print(f"Successfully processed participant {pid}")
        except Exception as e:
            print(f"Error processing participant {pid}: {str(e)}")
            failed_pids.append((pid, str(e)))
            continue
    
    print("\nProcessing complete!")
    print(f"Successfully processed {len(results)} participants")
    
    if failed_pids:
        print(f"\nFailed to process {len(failed_pids)} participants:")
        for pid, error in failed_pids:
            print(f"Participant {pid}: {error}")
            
    return results



ModuleNotFoundError: No module named 'pywt'

In [ ]:
# Process all participants
# sessions = [0, 1, 4]  # From session_mapping

# results = process_all_participants(sessions)

Processing participants:   0%|          | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_572905/483984249.py:82: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['X_L'] = df['X_L'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_572905/483984249.py:83: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Y_L'] = df['Y_L'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_572905/483984249.py:96: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  velocities = velocities.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_572905/483984249.py:230: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  left_pupil = trial_data['PupilSize

Processing participant 8


/tmp/ipykernel_572905/483984249.py:82: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['X_L'] = df['X_L'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_572905/483984249.py:83: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Y_L'] = df['Y_L'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_572905/483984249.py:96: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  velocities = velocities.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_572905/483984249.py:230: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  left_pupil = trial_data['PupilSize_L'].fillna(method='ffill').values - left_baseline
/tmp/ipykern

Error processing participant 8: single positional indexer is out-of-bounds
Processing participant 10


/tmp/ipykernel_572905/483984249.py:230: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  left_pupil = trial_data['PupilSize_L'].fillna(method='ffill').values - left_baseline
/tmp/ipykernel_572905/483984249.py:231: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  right_pupil = trial_data['PupilSize_R'].fillna(method='ffill').values - right_baseline
/tmp/ipykernel_572905/483984249.py:82: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['X_L'] = df['X_L'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_572905/483984249.py:83: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Y_L'] = df['Y_L'].fillna(method='ffill').fillna(method='bf

...

Error processing participant 31: single positional indexer is out-of-bounds
Processing participant 32
Error processing participant 32: single positional indexer is out-of-bounds


/tmp/ipykernel_572905/483984249.py:82: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['X_L'] = df['X_L'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_572905/483984249.py:83: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Y_L'] = df['Y_L'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_572905/483984249.py:96: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  velocities = velocities.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_572905/483984249.py:230: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  left_pupil = trial_data['PupilSize_L'].fillna(method='ffill').values - left_baseline
/tmp/ipykern

Processing participant 33


/tmp/ipykernel_572905/483984249.py:82: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['X_L'] = df['X_L'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_572905/483984249.py:83: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Y_L'] = df['Y_L'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_572905/483984249.py:96: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  velocities = velocities.fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_572905/483984249.py:230: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  left_pupil = trial_data['PupilSize_L'].fillna(method='ffill').values - left_baseline
/tmp/ipykern

Error processing participant 33: single positional indexer is out-of-bounds

Processing complete!
Successfully processed 1 participants

Failed to process 19 participants:
Participant 8: single positional indexer is out-of-bounds
Participant 10: single positional indexer is out-of-bounds
Participant 11: single positional indexer is out-of-bounds
Participant 13: single positional indexer is out-of-bounds
Participant 14: single positional indexer is out-of-bounds
Participant 15: single positional indexer is out-of-bounds
Participant 17: single positional indexer is out-of-bounds
Participant 19: single positional indexer is out-of-bounds
Participant 21: single positional indexer is out-of-bounds
Participant 22: single positional indexer is out-of-bounds
Participant 24: single positional indexer is out-of-bounds
Participant 25: single positional indexer is out-of-bounds
Participant 26: single positional indexer is out-of-bounds
Participant 27: single positional indexer is out-of-bounds
Par

In [ ]:
# View results
# results

{8:      leftPupilSize  rightPupilSize  fixationDur_mean  fixationDur_var  \
 0      1408.968788     1391.860059          0.194000         0.000000   
 1      1374.494406     1346.876928          0.147500         0.011772   
 2      1260.087508     1208.979422          0.677000         0.109561   
 3      1605.421417     1529.761276          0.228500         0.062192   
 4      1538.450481     1505.174165          0.157714         0.044250   
 ..             ...             ...               ...              ...   
 195    1298.485989     1389.815834          0.000000         0.000000   
 196    1268.257765     1303.297772          0.000000         0.000000   
 197    1303.946372     1325.514450          0.916000         0.000000   
 198    1436.240716     1516.593011          0.000000         0.000000   
 199    1896.116362     1875.091954          0.298000         0.000000   
 
      fixationDur_std  fixationDur_total  fixationCount  fixationsRate  \
 0           0.000000            

In [ ]:
# Combine results and save
# df = pd.concat(results.values(), keys=results.keys(), names=['SessionID'])
# df = df.reset_index()
# df.to_csv("./results/eye_tracking_features.csv", index=False)

## Fixations quality check

In [ ]:
# Fixation quality check
# print("Zero Fixation Counts by Condition:")
# zero_by_condition = df[df['fixationCount'] == 0].groupby('Condition').size()
# print(zero_by_condition)
# print("\nTotal rows by Condition:")
# print(df.groupby('Condition').size())

# Percentage of zeros in each condition
# print("\nPercentage of Zero Fixation Rows by Condition:")
# condition_total = df.groupby('Condition').size()
# zero_percentage = (zero_by_condition / condition_total * 100).round(2)
# print(zero_percentage)

# Check zeros by task count
# print("\nZero Fixation Counts by Task Count:")
# zero_by_task = df[df['fixationCount'] == 0].groupby('TaskCount').size()
# print(zero_by_task)

# Investigate potential patterns in zero fixations
# zero_fixation_df = df[df['fixationCount'] == 0]

# Check pupil sizes when fixation count is zero
# print("\nPupil Sizes for Zero Fixation Rows:")
# print("Left Pupil Size:")
# print(zero_fixation_df['leftPupilSize'].describe())
# print("\nRight Pupil Size:")
# print(zero_fixation_df['rightPupilSize'].describe())

# Compare with non-zero fixation rows
# non_zero_fixation_df = df[df['fixationCount'] > 0]

# print("\nComparison of Pupil Sizes:")
# print("Zero Fixation Left Pupil Mean:", zero_fixation_df['leftPupilSize'].mean())
# print("Non-Zero Fixation Left Pupil Mean:", non_zero_fixation_df['leftPupilSize'].mean())
# print("Zero Fixation Right Pupil Mean:", zero_fixation_df['rightPupilSize'].mean())
# print("Non-Zero Fixation Right Pupil Mean:", non_zero_fixation_df['rightPupilSize'].mean())

# Time distribution of zero fixation rows
# print("\nTime to First Fixation for Zero Fixation Rows:")
# print(zero_fixation_df['time2FirstFixation'].describe())

Zero Fixation Counts by Condition:
Condition
set1     73
set10    64
set11    78
set12    78
set13    65
set14    82
set15    67
set16    75
set17    83
set18    83
set19    78
set2     96
set20    73
set3     65
set4     87
set5     68
set6     72
set7     74
set8     83
set9     63
dtype: int64

Total rows by Condition:
Condition
set1     200
set10    200
set11    200
set12    200
set13    200
set14    200
set15    200
set16    200
set17    200
set18    200
set19    200
set2     200
set20    200
set3     200
set4     200
set5     200
set6     200
set7     200
set8     200
set9     200
dtype: int64

Percentage of Zero Fixation Rows by Condition:
Condition
set1     36.5
set10    32.0
set11    39.0
set12    39.0
set13    32.5
set14    41.0
set15    33.5
set16    37.5
set17    41.5
set18    41.5
set19    39.0
set2     48.0
set20    36.5
set3     32.5
set4     43.5
set5     34.0
set6     36.0
set7     37.0
set8     41.5
set9     31.5
dtype: float64

Zero Fixation Counts by Task Count:
Tas